In [50]:
import requests
import pandas as pd
import json
from tqdm import tqdm

In [36]:
BASE_URL = "http://localhost:8080/"
EMBEDDIGS_URL = BASE_URL + "songs/data/embeddings"

In [28]:


with open("data/all_in_one_playlist_dataset.json", "r") as file:
    data = json.load(file)
df = pd.DataFrame(data)

df = df.dropna()

In [55]:
def create_batches(tracks, N: int):
    batches = []
    batch = []
    
    for i, uri in enumerate(tracks):
        
        if i % N == 0 and i!=0:
            batches.append(batch)
            batch = []
        batch.append(uri)

    return(batches)        

batches = create_batches(df["track_uri"], 128)

In [54]:
out_data = {}
i = 0 
for batch in tqdm(batches):
    response = requests.post(EMBEDDIGS_URL, data=json.dumps({"track_uris": batch}))
    embeddings = response.json()["embeddings"]
    out_data |= {key: value for key, value in zip(batch, embeddings)}

    if i % 10 == 0:
        with open(f"data/out_data_temp_{i}.json", "w") as file:
            json.dump(out_data, file)
    i += 1

with open("data/out_data.json", "w") as file:
    json.dump(out_data, file)


  0%|          | 5/5638 [01:10<22:07:41, 14.14s/it]


KeyboardInterrupt: 